In [ ]:
!pip install -q -r requirements.txt

In [ ]:
!apt-get update >/dev/null
!apt-get -y install python3-dev libsm-dev libxrender1 libxext6 zip git >/dev/null
!rm -rf /var/lib/apt/lists/*

In [ ]:
!git pull

In [ ]:
import glob
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

from train import create_model, create_criterion, evaluate
from dataset import TrainData, TrainDataset

In [ ]:
input_dir = "/storage/kaggle/quickdraw"
image_size = 32
batch_size = 256
test_size = 0.1
train_on_unrecognized = True
num_categories_restriction = 68
num_workers = 8
pin_memory = True
use_dummy_image = False

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = create_model("cnn", input_size=image_size, num_classes=68)
model.load_state_dict(torch.load("/storage/models/quickdraw/test/model.pth", map_location=device))

In [ ]:
train_data = TrainData(
    input_dir,
    shard=0,
    test_size=test_size,
    train_on_unrecognized=train_on_unrecognized,
    num_categories_restriction=num_categories_restriction)

In [ ]:
val_set = TrainDataset(train_data.val_set_df, image_size, use_dummy_image)
val_set_data_loader = \
    DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

In [ ]:
criterion = create_criterion("cce", len(train_data.categories))

In [ ]:
loss_avg, mapk_avg, accuracy_top1_avg, accuracy_top3_avg, accuracy_top5_avg, accuracy_top10_avg = \
    evaluate(model, val_set_data_loader, criterion, 3)

In [ ]:
print(
    "loss: {:.2f}, map@3: {:.2f}, acc@1: {:.2f}, acc@3: {:.2f}, acc@5: {:.2f}, acc@10: {:.2f}"
    .format(loss_avg, mapk_avg, accuracy_top1_avg, accuracy_top3_avg, accuracy_top5_avg, accuracy_top10_avg))

In [ ]:
val_set_data_loader_iter = iter(val_set_data_loader)

In [ ]:
batch = next(val_set_data_loader_iter)

images, categories = \
    batch[0].to(device, non_blocking=True), \
    batch[1].to(device, non_blocking=True)

In [ ]:
prediction_logits = model(images)
predictions = F.softmax(prediction_logits, dim=1)

In [ ]:
prediction_scores, prediction_categories = predictions.topk(3, dim=1, sorted=True)

In [ ]:
print(prediction_scores[:10, 0])

In [ ]:
print(prediction_scores[:, 0].mean())
print(prediction_scores[:, 0].std())
print(prediction_scores[:, 0].min())
print(prediction_scores[:, 0].max())